# Imports

In [1]:
from sat import *
from px import *
from run_px_experiments import parse_output

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

#PROBLEM_FILE = './toy_sat_problems/toy_sat1.cnf'
#SOLUTIONS_FILE = './toy_sat_problems/toy_sat_1_solutions.txt'

#PROBLEM_FILE = './toy_sat_problems/toy_sat2.cnf'
#SOLUTIONS_FILE = './toy_sat_problems/toy_sat_2_solutions.txt'

#PROBLEM_FILE = './toy_sat_problems/toy_sat3.cnf'
#SOLUTIONS_FILE = './toy_sat_problems/toy_sat_3_solutions.txt'

#PROBLEM_FILE = './sat_problems/test_folder/001-80-12.cnf'
#SOLUTIONS_FILE = './results/ubcsat_outputs/001-80-12.txt'

#PROBLEM_FILE = './sat_problems/trial_problems/LABS_n038_goal002.cnf'
#SOLUTIONS_FILE = './results/ubcsat_outputs/LABS_n038_goal002.txt'

PROBLEM_FILE = './sat_problems/trial_problems/002-80-12.cnf'
SOLUTIONS_FILE = './results/ubcsat_outputs/002-80-12.txt'

In [2]:
def draw_graph(graph):
    nx.draw(graph, with_labels=True, node_color='white', node_size=400)
    ax = plt.gca() # to get the current axis
    ax.collections[0].set_edgecolor("#000000") 

# Load Problems / Solutions

In [3]:
%%time
sat = read_sat_problem(PROBLEM_FILE)

#print('Clause #, Variables')
#for i in range(len(sat.clauses)):
#    print(i, sat.clauses[i])

#print('\nVariable #, Clauses')
#sat.create_var_to_clause_dict()
#print(sat.var_to_clause_dict)

#print('\nUnique Variables')
#print(sat.unique_vars)

sat

Wall time: 4.83 s


SAT(n = 13408, m = 478488, k = 8, name = '002-80-12.cnf')

In [4]:
%%time
solutions = parse_output(SOLUTIONS_FILE)

print(solutions.shape)

p1 = solutions[6]
p2 = solutions[7]

p1_unsat = np.argwhere(~(sat.evaluate_solution(p1))).flatten()
p2_unsat = np.argwhere(~(sat.evaluate_solution(p2))).flatten()
print(f'P1 Unsat: {len(p1_unsat)}')
print(f'P2 Unsat: {len(p2_unsat)}')

print('')
print(f'P1 Score: {sat.score_solution(p1)}')
print(f'P2 Score: {sat.score_solution(p2)}')

(20, 13408)
P1 Unsat: 1013
P2 Unsat: 1013

P1 Score: 477475
P2 Score: 477475
Wall time: 2.63 s


In [5]:
p1_unsat

array([    18,    116,    304, ..., 477639, 478102, 478480], dtype=int64)

In [6]:
p2_unsat

array([   128,    157,    184, ..., 478316, 478341, 478479], dtype=int64)

In [7]:
set(p1_unsat).intersection(set(p2_unsat))

{6330, 14176, 210618, 221789, 249691, 337799, 460194}

In [8]:
"""with open(SOLUTIONS_FILE) as solutions_file:
    solutions = [line.strip() for line in solutions_file.readlines()]
    
print(f'P1 Bitstring: {solutions[0]}')
print(f'P2 Bitstring: {solutions[1]}')
print('')

p1 = bitstring_to_int_array(solutions[0])
p2 = bitstring_to_int_array(solutions[1])
print(f'P1 Assignments: {p1}')
print(f'P2 Assignments: {p2}')
print('')

p1_unsat = np.argwhere(~(sat.evaluate_solution(p1))).flatten()
p2_unsat = np.argwhere(~(sat.evaluate_solution(p2))).flatten()
print(f'P1 Unsat Clauses: {p1_unsat}, {sat.clauses[p1_unsat]}')
print(f'P2 Unsat Clauses: {p2_unsat}, {sat.clauses[p2_unsat]}')
print('')

print(f'P1 Score: {sat.score_solution(p1)}')
print(f'P2 Score: {sat.score_solution(p2)}')"""

"with open(SOLUTIONS_FILE) as solutions_file:\n    solutions = [line.strip() for line in solutions_file.readlines()]\n    \nprint(f'P1 Bitstring: {solutions[0]}')\nprint(f'P2 Bitstring: {solutions[1]}')\nprint('')\n\np1 = bitstring_to_int_array(solutions[0])\np2 = bitstring_to_int_array(solutions[1])\nprint(f'P1 Assignments: {p1}')\nprint(f'P2 Assignments: {p2}')\nprint('')\n\np1_unsat = np.argwhere(~(sat.evaluate_solution(p1))).flatten()\np2_unsat = np.argwhere(~(sat.evaluate_solution(p2))).flatten()\nprint(f'P1 Unsat Clauses: {p1_unsat}, {sat.clauses[p1_unsat]}')\nprint(f'P2 Unsat Clauses: {p2_unsat}, {sat.clauses[p2_unsat]}')\nprint('')\n\nprint(f'P1 Score: {sat.score_solution(p1)}')\nprint(f'P2 Score: {sat.score_solution(p2)}')"

# PX

In [9]:
%%time
vig = get_vig(sat)

print(vig.number_of_nodes(), 
      vig.number_of_edges(), 
      nx.number_connected_components(vig))

##draw_graph(vig)

13408 150540 1
Wall time: 15.2 s


In [10]:
%%time
decomposed_vig = decompose_vig(vig, p1, p2)

print(decomposed_vig.number_of_nodes(), 
      decomposed_vig.number_of_edges(), 
      nx.number_connected_components(decomposed_vig))

#draw_graph(decomposed_vig)

6499 32270 2
Wall time: 427 ms


In [11]:
%%time
new_solution = partition_crossover(sat, decomposed_vig, p1, p2, none_fill='p1', verbose=2)

print('')
print(f'New Solution: {new_solution}')
print(f'New Solution Score: {sat.score_solution(new_solution)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution)]}')

Common variable assignments: [1 2 3 ... -13406 13407 13408]
Component: {5, 7, 10, 11, 15, 16, 17, 18, 20, 21, 22, 26, 27, 28, 30, 32, 33, 34, 36, 40, 44, 45, 47, 48, 49, 53, 54, 56, 62, 64, 69, 70, 71, 72, 73, 78, 79, 81, 84, 86, 90, 91, 92, 94, 99, 101, 104, 105, 107, 111, 116, 117, 121, 122, 124, 125, 126, 127, 128, 130, 136, 137, 140, 141, 143, 144, 145, 146, 147, 148, 151, 152, 154, 155, 156, 157, 159, 160, 161, 162, 163, 168, 169, 170, 172, 173, 174, 177, 178, 179, 180, 183, 184, 186, 187, 189, 191, 193, 196, 199, 202, 203, 204, 206, 207, 209, 210, 212, 213, 214, 215, 216, 217, 220, 221, 222, 223, 228, 231, 233, 238, 240, 241, 242, 245, 249, 252, 254, 257, 258, 261, 262, 263, 266, 267, 268, 269, 271, 272, 273, 275, 278, 280, 282, 287, 288, 289, 293, 295, 296, 299, 300, 303, 305, 306, 308, 309, 310, 311, 314, 317, 318, 319, 321, 322, 324, 328, 330, 331, 332, 333, 334, 336, 340, 341, 343, 344, 346, 349, 353, 355, 356, 357, 358, 361, 362, 365, 366, 368, 369, 371, 372, 373, 374, 379, 

	Sub problem clauses: [{-513, -417, -257, -1, -65} {513, 417, -257, -1, -65}
 {513, 257, -417, -1, -65} ... {-5311, -2717, -13404, 2877} {-2820}
 {-2771}]
	P1 Score: 459835, P2 Score: 459836
Component: {13393, 2865, 2866}
	Sub problem clauses: [{-13392, 2705, -5299, -2865} {-13392, -2865, 5299, -2705}
 {13392, 2705, -5299, 2865} {13392, 2865, 5299, -2705}
 {13392, -2865, -13393} {-13392, 13393, 2865} {2705, 5299, -13393}
 {13393, -5299, -2705} {13392, 2705, 5299, -2865}
 {-13392, 2865, -5299, -2705} {2706, -5300, -2866, -13393}
 {5300, -2866, -2706, -13393} {13393, 2706, -5300, 2866}
 {13393, 2866, 5300, -2706} {13393, -2866, -13394} {13394, 2866, -13393}
 {13393, 2706, 5300, -2866} {2866, -5300, -2706, -13393}]
	P1 Score: 18, P2 Score: 17
Solution after recombination: [1 2 3 ... -13406 13407 13408]
Filling in None spots with assignments from P1

New Solution: [1 2 3 ... -13406 13407 13408]
New Solution Score: 477476
Remaining Unsatisfied Clauses: [{-521, -425, -265, -9, -73} {426, 522

# PX Prime on P1

In [12]:
%%time
decomposed_sat, iterations = decompose_problem(sat, p1, p2, p1_unsat, p2_unsat, init_method='p1', verbose=1)

print('')
#print(decomposed_sat.clauses)
#print(decomposed_sat.unique_vars)

decomposed_sat

[3,4] Length of Init SWAPC: 1006
[5] Length of Init VAR: 4423
[5] Length of No Common Variables VAR: 2146
Length of sat_by_common: 420760
[6] Length of SWAPC: 309099
[7] Length of SWAPC: 43934
[8] Length of VAR: 6040
[6] Length of SWAPC: 454781
[7] Length of SWAPC: 57461
[8] Length of VAR: 6466
[6] Length of SWAPC: 460726
[7] Length of SWAPC: 57704
[8] Length of VAR: 6493
[6] Length of SWAPC: 460839
[7] Length of SWAPC: 57717
[8] Length of VAR: 6496
[6] Length of SWAPC: 460841
[7] Length of SWAPC: 57717
[8] Length of VAR: 6496
Loop ran 5 times

Wall time: 4.48 s


SAT(n = 6496, m = 57717, k = 8, name = 'decomposed_002-80-12.cnf')

In [13]:
%%time
vig_prime = get_vig(decomposed_sat, verbose=False)

print(vig_prime.number_of_nodes(), 
      vig_prime.number_of_edges(), 
      nx.number_connected_components(vig_prime))

#draw_graph(vig_prime)

6496 32226 1
Wall time: 652 ms


In [14]:
%%time
decomposed_vig_prime = decompose_vig(vig_prime, p1, p2)

print(decomposed_vig_prime.number_of_nodes(), 
      decomposed_vig_prime.number_of_edges(), 
      nx.number_connected_components(decomposed_vig_prime))

#draw_graph(decomposed_vig_prime)

6496 32226 1
Wall time: 106 ms


In [20]:
%%time
new_solution_prime = partition_crossover(decomposed_sat, decomposed_vig_prime, p1, p2, none_fill='p1', verbose=2)

print('')
print(f'New Solution: {new_solution_prime}')
print(f'New Solution Score: {sat.score_solution(new_solution_prime)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution_prime)]}')


New Solution: [     1      2      3 ... -13406  13407  13408]
New Solution Score: 477475
Remaining Unsatisfied Clauses: [{-521, -425, -265, -9, -73} {426, 522, 10, 74, -266}
 {12, 524, -76, -268, -428} ... {-13392, 2705, -5299, -2865}
 {2867, 13395, -13394} {-2820}]
Wall time: 1.5 s


In [16]:
old_method_unsat = set(np.argwhere(~sat.evaluate_solution(new_solution)).flatten().tolist())
new_method_unsat = set(np.argwhere(~sat.evaluate_solution(new_solution_prime)).flatten().tolist())
clauses_of_interest = list(new_method_unsat - old_method_unsat)
print(clauses_of_interest)
print(sat.clauses[clauses_of_interest])

vars_of_interest = get_unique_vars(sat.clauses[clauses_of_interest])
print(vars_of_interest)

print(f'P1  Assignments: {get_assignments(p1, vars_of_interest)}')
print(f'P2  Assignments: {get_assignments(p2, vars_of_interest)}')
print(f'NS  Assignments: {get_assignments(new_solution, vars_of_interest)}')
print(f'NS* Assignments: {get_assignments(new_solution_prime, vars_of_interest)}')

[478316]
[{-13392, 2705, -5299, -2865}]
{13392, 2705, 5299, 2865}
P1  Assignments: {13392, -2865, 5299, -2705}
P2  Assignments: {13392, 2865, 5299, -2705}
NS  Assignments: {13392, -2865, 5299, -2705}
NS* Assignments: {13392, 2865, 5299, -2705}


In [19]:
get_unique_vars(sat.clauses[list(sat.clauses_with_variables([2865]))])

{2705, 2865, 5299, 13392, 13393}

# PX Prime on P2

In [ ]:
%%time
decomposed_sat, iterations = decompose_problem(sat, p1, p2, p1_unsat, p2_unsat, init_method='p2', verbose=1)

#print('')
#print(decomposed_sat.clauses)
#print(decomposed_sat.unique_vars)

decomposed_sat

In [ ]:
%%time
vig_prime = get_vig(decomposed_sat, verbose=False)

print(vig_prime.number_of_nodes(), 
      vig_prime.number_of_edges(), 
      nx.number_connected_components(vig_prime))

#draw_graph(vig_prime)

In [ ]:
%%time
decomposed_vig_prime = decompose_vig(vig_prime, p1, p2)

print(decomposed_vig_prime.number_of_nodes(), 
      decomposed_vig_prime.number_of_edges(), 
      nx.number_connected_components(decomposed_vig_prime))

#draw_graph(decomposed_vig_prime)

In [ ]:
%%time
new_solution_prime = partition_crossover(decomposed_sat, decomposed_vig_prime, p1, p2, none_fill='p2', verbose=2)

print('')
print(f'New Solution: {new_solution_prime}')
print(f'New Solution Score: {sat.score_solution(new_solution_prime)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution_prime)]}')

# PX Prime on XOR

In [ ]:
%%time
decomposed_sat, iterations = decompose_problem(sat, p1, p2, p1_unsat, p2_unsat, init_method='xor', verbose=1)

#print('')
#print(decomposed_sat.clauses)
#print(decomposed_sat.unique_vars)

decomposed_sat

In [ ]:
%%time
vig_prime = get_vig(decomposed_sat, verbose=False)

print(vig_prime.number_of_nodes(), 
      vig_prime.number_of_edges(), 
      nx.number_connected_components(vig_prime))

#draw_graph(vig_prime)

In [ ]:
%%time
decomposed_vig_prime = decompose_vig(vig_prime, p1, p2)

print(decomposed_vig_prime.number_of_nodes(), 
      decomposed_vig_prime.number_of_edges(), 
      nx.number_connected_components(decomposed_vig_prime))

#draw_graph(decomposed_vig_prime)

In [ ]:
%%time
new_solution_prime = partition_crossover(decomposed_sat, decomposed_vig_prime, p1, p2, none_fill='p1', verbose=2)

print('')
print(f'New Solution: {new_solution_prime}')
print(f'New Solution Score: {sat.score_solution(new_solution_prime)}')
print(f'Remaining Unsatisfied Clauses: {sat.clauses[~sat.evaluate_solution(new_solution_prime)]}')